# SimpleForwardModelForTransmissionMeasurement
- author  : Sylvie Dagoret‑Campagne
- creation date : 2025-11-19
- last update : 2025-11-20

In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from functools import partial

In [ ]:
plt.rcParams["figure.figsize"] = (8,6)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'
plt.rcParams['legend.fontsize'] = 'xx-large'

#mpl.rcParams['font.size'] = 16       # global font size
#mpl.rcParams['axes.labelsize'] = 18  # axis label size
#mpl.rcParams['axes.titlesize'] = 20  # title size
#mpl.rcParams['legend.fontsize'] = 14
#mpl.rcParams['xtick.labelsize'] = 13
#mpl.rcParams['ytick.labelsize'] = 13

In [ ]:
# where are stored the figures
pathfigs = "figs_SimpleForwardModelTransm"
prefix = "fwmodel_transm"
if not os.path.exists(pathfigs):
    os.makedirs(pathfigs) 
figtype = ".png"

In [ ]:
kk= 0.04

In [ ]:
def f(x,alpha,k=kk):
    """
    """
    return np.exp(-k*np.power(x,alpha))
def g(x,alpha,k=kk):
    """
    """
    return k*np.power(x,alpha)

In [ ]:
def dfdx(x,alpha,k=kk):
    """
    """
    return -alpha*k*np.power(x,alpha-1)*np.exp(-k*np.power(x,alpha))
def dgdx(x,alpha,k=kk):
    """
    """
    return alpha*k*np.power(x,alpha-1)

In [ ]:
def dxdf(x,alpha,k=kk):
    """
    """
    return 1/dfdx(x,alpha,k)
def dxdg(x,alpha,k=kk):
    """
    """
    return 1/dgdx(x,alpha,k)

In [ ]:
params_alpha = [1.5,1,0.5]
params_color = ["b","k","r"]
params_labels = [r"$\alpha$ = 1.5", r"$\alpha$ = 1", r"$\alpha$ = 1/2" ]


nparams = len(params_alpha)

In [ ]:
f_list = [lambda x, a=a: f(x, a) for a in params_alpha]
g_list = [lambda x, a=a: g(x, a) for a in params_alpha]
dfdx_list = [lambda x, a=a: dfdx(x, a) for a in params_alpha]
dgdx_list = [lambda x, a=a: dgdx(x, a) for a in params_alpha]
dfdxabs_list = [lambda x, a=a: np.abs(dfdx(x, a)) for a in params_alpha]
dgdxabs_list = [lambda x, a=a: np.abs(dgdx(x, a)) for a in params_alpha]
dfdx_inv_abs_list = [lambda x, a=a: 1./np.abs(dfdx(x, a)) for a in params_alpha]
dgdx_inv_abs_list = [lambda x, a=a: 1./np.abs(dgdx(x, a)) for a in params_alpha]

In [ ]:
f_list = [partial(f, alpha=a) for a in params_alpha]
g_list = [partial(g, alpha=a) for a in params_alpha]

dfdx_list = [partial(dfdx,alpha=a) for a in params_alpha]
dgdx_list = [partial(dgdx,alpha=a) for a in params_alpha]

dxdf_list = [partial(dxdf,alpha=a) for a in params_alpha]
dxdg_list = [partial(dxdg,alpha=a) for a in params_alpha]

In [ ]:
x= np.linspace(0.04,20)

In [ ]:
#---------------------------------------------
# 1) Organize content for each column
#---------------------------------------------

columns = [
    {
        "eq1": r"$y = f(x)$",
        "eq2": r"$f(x) = e^{-k\cdot x^\alpha}$",
        "func_list": f_list,
        "ylabel": "y = f(x)",
    },
    {
        "eq1": r"$y = |\frac{df}{dx}(x)|$",
        "eq2": r"$ \frac{\sigma_y}{\sigma_x} = \alpha \cdot k \cdot x^{\alpha-1}e^{-k\cdot x^\alpha} $",
        "func_list": dfdx_list,
        "ylabel": r"$y = |df/dx(x)|$",
    },
    {
        "eq1": r"$y = 1/|\frac{df}{dx}(x)|$",
        "eq2": r"$\frac{\sigma_x}{\sigma_y} = \frac{\cdot x^{1-\alpha}}{\alpha \cdot k} e^{k\cdot x^\alpha}$",
        "func_list": dxdf_list,
        "ylabel": r"$y = 1/|df/dx(x)|$",
    }
]

# For the second row of equations + plots (g, dg/dx, ...)
columns2 = [
    {
        "eq1": r"$y_m = g(x)$",
        "eq2": r"$y_m = - \ln f(x) = k x^\alpha$",
        "func_list": g_list,
        "ylabel": "y = g(x)",
    },
    {
        "eq1": r"$y = |\frac{dg}{dx}(x)|$",
        "eq2": r"$ \frac{\sigma_{ym}}{\sigma_x} = \alpha \cdot k \cdot x^{\alpha-1}$",
        "func_list": dgdx_list,
        "ylabel": r"$y = |dg/dx(x)|$",
    },
    {
        "eq1": r"$y = 1/|\frac{dg}{dx}(x)|$",
        "eq2": r"$\frac{\sigma_x}{\sigma_{ym}} = \frac{x^{1-\alpha}}{\alpha \cdot k}  $",
        "func_list": dxdg_list,
        "ylabel": r"$y = 1/|dg/dx(x)|$",
    }
]




#---------------------------------------------
# 2) Create figure
#---------------------------------------------
fig = plt.figure(figsize=(16, 10), layout="constrained")
gs = GridSpec(nrows=4, ncols=3, figure=fig, hspace=0.1, wspace=0.2,  
              height_ratios=[0.5, 2, 0.5, 2])

axes = []

#---------------------------------------------
# 3) Fill the grid using loops
#---------------------------------------------
for col, entry in enumerate(columns):
    # Equation (row 0)
    ax_eq = fig.add_subplot(gs[0, col])
    ax_eq.axis("off")
    ax_eq.text(0.5, 0.55, entry["eq1"], fontsize=20, ha="center")
    ax_eq.text(0.5, 0.1, entry["eq2"], fontsize=20, ha="center")

    # Plot (row 1)
    ax = fig.add_subplot(gs[1, col])
    axes.append(ax)
    for idx, func in enumerate(entry["func_list"]):
        ax.plot(x, np.abs(func(x)), color=params_color[idx],
                label=params_labels[idx])
    ax.set_xlabel("x")
    ax.set_ylabel(entry["ylabel"])
    ax.legend()

# Second row of equations and plots
for col, entry in enumerate(columns2):
    # Equation (row 2)
    ax_eq = fig.add_subplot(gs[2, col])
    ax_eq.axis("off")
    ax_eq.text(0.5, 0.55, entry["eq1"], fontsize=20, ha="center")
    ax_eq.text(0.5, 0.1, entry["eq2"], fontsize=20, ha="center")

    # Plot (row 3)
    ax = fig.add_subplot(gs[3, col])
    axes.append(ax)
    for idx, func in enumerate(entry["func_list"]):
        ax.plot(x, np.abs(func(x)), color=params_color[idx],
                label=params_labels[idx])
    ax.set_xlabel("x")
    ax.set_ylabel(entry["ylabel"])
    ax.legend()

#---------------------------------------------
# 4) Hide ticks everywhere
#---------------------------------------------
for ax in axes:
    ax.tick_params(
        axis='both', which='both',
        bottom=False, top=False,
        left=False, right=False,
        labelbottom=True, labelleft=False,
    )

plt.suptitle(
    "Basic Forward model of flux attenuation and error on x estimation",
    fontsize=20, fontweight="bold"
)

figname =f"{pathfigs}/{prefix}_SimpleForwardModelForTrnasmission_nolabel"+figtype
plt.savefig(figname)

plt.show()


## Adding insets

In [ ]:
from mpl_toolkits.axes_grid1.inset_locator import inset_axes, mark_inset

In [ ]:
#---------------------------------------------
# 1) Organize content for each column
#---------------------------------------------
columns = [
    {
        "eq1": r"$y = f(x)$",
        "eq2": r"$f(x) = e^{-k\cdot x^\alpha}$",
        "func_list": f_list,
        "ylabel": "y = f(x)",
    },
    {
        "eq1": r"$y = |\frac{df}{dx}(x)|$",
        "eq2": r"$ \frac{\sigma_y}{\sigma_x} = \alpha \cdot k \cdot x^{\alpha-1}e^{-k\cdot x^\alpha} $",
        "func_list": dfdx_list,
        "ylabel": r"$y = |df/dx(x)|$",
    },
    {
        "eq1": r"$y = 1/|\frac{df}{dx}(x)|$",
        "eq2": r"$\frac{\sigma_x}{\sigma_y} = \frac{\cdot x^{1-\alpha}}{\alpha \cdot k} e^{k\cdot x^\alpha}$",
        "func_list": dxdf_list,
        "ylabel": r"$y = 1/|df/dx(x)|$",
    }
]

# For the second row of equations + plots (g, dg/dx, ...)
columns2 = [
    {
        "eq1": r"$y_m = g(x)$",
        "eq2": r"$y_m = - \ln f(x) = k x^\alpha$",
        "func_list": g_list,
        "ylabel": "y = g(x)",
    },
    {
        "eq1": r"$y = |\frac{dg}{dx}(x)|$",
        "eq2": r"$ \frac{\sigma_{ym}}{\sigma_x} = \alpha \cdot k \cdot x^{\alpha-1}$",
        "func_list": dgdx_list,
        "ylabel": r"$y = |dg/dx(x)|$",
    },
    {
        "eq1": r"$y = 1/|\frac{dg}{dx}(x)|$",
        "eq2": r"$\frac{\sigma_x}{\sigma_{ym}} = \frac{x^{1-\alpha}}{\alpha \cdot k}  $",
        "func_list": dxdg_list,
        "ylabel": r"$y = 1/|dg/dx(x)|$",
    }
]

#---------------------------------------------
# 2) Create figure
#---------------------------------------------
fig = plt.figure(figsize=(16, 10), layout="constrained")
gs = GridSpec(nrows=4, ncols=3, figure=fig, hspace=0.1, wspace=0.2,  
              height_ratios=[0.75, 2, 0.75, 2])

axes = []
axes_keep = []

#---------------------------------------------
# 3) Fill the grid using loops
#---------------------------------------------
for col, entry in enumerate(columns):
    # Equation (row 0)
    ax_eq = fig.add_subplot(gs[0, col])
    ax_eq.axis("off")
    ax_eq.text(0.5, 0.55, entry["eq1"], fontsize=20, ha="center")
    ax_eq.text(0.5, 0.1, entry["eq2"], fontsize=20, ha="center")

    # Plot (row 1)
    ax = fig.add_subplot(gs[1, col])
    axes.append(ax)
    if col==0:
        axes_keep.append(ax)
    for idx, func in enumerate(entry["func_list"]):
        ax.plot(x, np.abs(func(x)), color=params_color[idx],
                label=params_labels[idx])
    ax.set_xlabel("x")
    #ax.set_ylabel(entry["ylabel"])

    # SHOW Y LABEL ONLY FOR LEFTMOST SUBPLOT
    if col == 0:
        ax.set_ylabel(entry["ylabel"])
    else:
        ax.set_ylabel("")
    ax.legend()
    
    # Create inset ONLY for the first column (col == 0)
    if col == 0:
        axins = inset_axes(
            ax,
            width="40%",            # relative size of inset
            height="40%",
            loc='lower right',      # position inside main axes
            borderpad=1
        )

        # Plot same curves inside inset
        for idx, func in enumerate(entry["func_list"]):
            axins.plot(x, np.abs(func(x)) ,color=params_color[idx])

        # Zoom window
        axins.set_xlim(0, 2.5)
        # Automatically set y-limits from data inside zoom
        ymin, ymax = np.inf, -np.inf
        for idx, func in enumerate(entry["func_list"]):
            y_zoom = np.abs(func(x[(x>=0) & (x<=2.5)]))
            ymin = min(ymin, y_zoom.min())
            ymax = max(ymax, y_zoom.max())
        axins.set_ylim(ymin, ymax)

        # Remove ticks inside inset
        axins.tick_params(bottom=True, labelbottom=True,
                      left=False, labelleft=False)

        # Draw a rectangle showing the zoom region
        #mark_inset(ax, axins, loc1=2, loc2=4, fc="none", ec="0.5")
    

# Second row of equations and plots
for col, entry in enumerate(columns2):
    # Equation (row 2)
    ax_eq = fig.add_subplot(gs[2, col])
    ax_eq.axis("off")
    ax_eq.text(0.5, 0.55, entry["eq1"], fontsize=20, ha="center")
    ax_eq.text(0.5, 0.1, entry["eq2"], fontsize=20, ha="center")

    # Plot (row 3)
    ax = fig.add_subplot(gs[3, col])
    axes.append(ax)
    #if col==0:
    #    axes_keep.append(ax)
    for idx, func in enumerate(entry["func_list"]):
        ax.plot(x, np.abs(func(x)), color=params_color[idx],
                label=params_labels[idx])
    ax.set_xlabel("x")
    ax.set_ylabel(entry["ylabel"])
    ax.legend()
    
    # Create inset ONLY for the first column (col == 0)
    if col == 0:
        axins = inset_axes(
            ax,
            width="40%",            # relative size of inset
            height="40%",
            loc='upper right',      # position inside main axes
            borderpad=1
        )

        # Plot same curves inside inset
        for idx, func in enumerate(entry["func_list"]):
            axins.plot(x, np.abs(func(x)) ,color=params_color[idx])

        # Zoom window
        axins.set_xlim(0, 2.5)
        # Automatically set y-limits from data inside zoom
        ymin, ymax = np.inf, -np.inf
        for idx, func in enumerate(entry["func_list"]):
            y_zoom = np.abs(func(x[(x>=0) & (x<=2.5)]))
            ymin = min(ymin, y_zoom.min())
            ymax = max(ymax, y_zoom.max())
        axins.set_ylim(ymin, ymax)

        # Remove ticks inside inset
        axins.tick_params(bottom=True, labelbottom=True,
                      left=False, labelleft=False)

        # Draw a rectangle showing the zoom region
        #mark_inset(ax, axins, loc1=2, loc2=4, fc="none", ec="0.5")
        

#---------------------------------------------
# 4) Hide ticks everywhere
#---------------------------------------------
for ax in axes:
    ax.tick_params(
        axis='both', which='both',
        bottom=True, top=False,
        left=True, right=False,
        labelbottom=True, labelleft=True,
    )

for ax in axes:
    ax.set_yticks([0])
    ax.set_yticklabels(["0"])
    ax.tick_params(axis='y', labelleft=True)

for ax in axes_keep:    
    ax.set_yticks([0, 1])
    ax.set_yticklabels(["0", "1"])
    ax.tick_params(axis='y', labelleft=True)

plt.suptitle(
    "Basic Forward model of flux attenuation and error on x estimation",
    fontsize=20, fontweight="bold"
)

figname =f"{pathfigs}/{prefix}_SimpleForwardModelForTrnasmission_wthlabelandinset"+figtype
plt.savefig(figname)

plt.show()
